In [58]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, LSTM, Dropout, concatenate, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder

In [161]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Rescaling
from tensorflow.keras import Model
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam

TensorFlow version: 2.11.0


In [203]:
data = pd.read_pickle('total_data_pickle')

In [204]:
labels = data["label"].values

In [205]:
values = data.drop(columns=['label'])

In [206]:
values = values.values

In [207]:
X_train, X_test, y_train, y_test = train_test_split(values,labels, test_size=0.2, random_state=42)

In [208]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [209]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (5471, 150, 1)
X_test shape: (1368, 150, 1)
y_train shape: (5471,)
y_test shape: (1368,)


In [210]:
nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

In [211]:
input_shape = X_train.shape[1:]

In [212]:
enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

In [213]:
input_shape

(150, 1)

In [214]:
(X_train.shape)

(5471, 150, 1)

In [189]:
padding = 'valid'
input_layer = keras.layers.Input(input_shape)

conv1 = keras.layers.Conv1D(filters=6,kernel_size=7,padding=padding,activation='sigmoid')(input_layer)
conv1 = keras.layers.AveragePooling1D(pool_size=3)(conv1)

conv2 = keras.layers.Conv1D(filters=12,kernel_size=7,padding=padding,activation='sigmoid')(conv1)
conv2 = keras.layers.AveragePooling1D(pool_size=3)(conv2)

flatten_layer = keras.layers.Flatten()(conv2)

output_layer = keras.layers.Dense(units=nb_classes,activation='softmax')(flatten_layer)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(),
                metrics=['accuracy'])

# model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
#                 metrics=['accuracy'])


In [218]:
model = Sequential()
model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(nb_classes, activation='softmax'))

# Compile the model
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

c:\Users\UTKU\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [215]:
from tensorflow.keras.layers import Concatenate, Input, Conv1D, MaxPooling1D, LSTM, GlobalMaxPooling1D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model

input_layer = Input(shape=input_shape)
conv_layer1 = Conv1D(filters=32, kernel_size=3, activation='relu')(input_layer)
conv_layer2 = Conv1D(filters=32, kernel_size=3, activation='relu')(conv_layer1)
maxpool_layer = MaxPooling1D(pool_size=2)(conv_layer2)
rnn_layer = LSTM(units=64, return_sequences=True)(maxpool_layer)
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(maxpool_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
cnn_layer = MaxPooling1D(pool_size=2)(cnn_layer)
cnn_layer = Flatten()(cnn_layer)
rnn_layer = Flatten()(rnn_layer)
rnn_layer = Dense(64)(rnn_layer)
combined_layer = Concatenate()([rnn_layer, cnn_layer])
global_pooling_layer = GlobalMaxPooling1D()(combined_layer[..., None])  # Add an extra dimension
dense_layer = Dense(units=128, activation='relu')(global_pooling_layer)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(units=nb_classes, activation='softmax')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [230]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, GlobalMaxPooling1D, concatenate

# Single input
input_layer = Input(shape=input_shape)

# Convolutional branch
conv_branch = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
conv_branch = MaxPooling1D(pool_size=2)(conv_branch)

# Recurrent branch
rnn_branch = LSTM(units=64, return_sequences=True)(input_layer)

# Apply global max pooling to both branches
conv_branch = GlobalMaxPooling1D()(conv_branch)
rnn_branch = GlobalMaxPooling1D()(rnn_branch)

# Concatenate the outputs of the convolutional and recurrent branches
concatenated = concatenate([conv_branch, rnn_branch])

# Classification layers
dense_layer = Dense(units=128, activation='relu')(concatenated)
output_layer = Dense(units=nb_classes, activation='softmax')(dense_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [235]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, GlobalMaxPooling1D, concatenate

# Single input
input_layer = Input(shape=input_shape)

# Convolutional branch
conv_branch = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
conv_branch = MaxPooling1D(pool_size=2)(conv_branch)

# First recurrent branch
rnn_branch1 = LSTM(units=64, return_sequences=True)(input_layer)

# Second recurrent branch
rnn_branch2 = LSTM(units=64, return_sequences=True)(input_layer)

# Apply global max pooling to the convolutional branch
conv_branch = GlobalMaxPooling1D()(conv_branch)

# Apply global max pooling to the recurrent branches
rnn_branch1 = GlobalMaxPooling1D()(rnn_branch1)
rnn_branch2 = GlobalMaxPooling1D()(rnn_branch2)

# Concatenate the outputs of all branches
concatenated = concatenate([conv_branch, rnn_branch1, rnn_branch2])

# Classification layers
dense_layer = Dense(units=128, activation='relu')(concatenated)
output_layer = Dense(units=nb_classes, activation='softmax')(dense_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [239]:
from tensorflow.keras.callbacks import EarlyStopping

batch_size = 32
nb_epochs = 500

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compile the model
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with optimizations
hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs,
                 validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/500
171/171 [==============================] - 11s 61ms/step - loss: 0.6772 - accuracy: 0.6077 - val_loss: 0.6930 - val_accuracy: 0.5965
Epoch 2/500
171/171 [==============================] - 11s 64ms/step - loss: 0.7601 - accuracy: 0.5924 - val_loss: 1.4550 - val_accuracy: 0.3918
Epoch 3/500
171/171 [==============================] - 11s 65ms/step - loss: 1.0331 - accuracy: 0.5357 - val_loss: 1.0166 - val_accuracy: 0.5914
Epoch 4/500
171/171 [==============================] - 11s 66ms/step - loss: 0.9328 - accuracy: 0.5710 - val_loss: 0.7791 - val_accuracy: 0.5892
Epoch 5/500
171/171 [==============================] - 12s 69ms/step - loss: 0.8781 - accuracy: 0.5778 - val_loss: 0.8023 - val_accuracy: 0.7449
Epoch 6/500
171/171 [==============================] - 12s 67ms/step - loss: 0.7416 - accuracy: 0.6200 - val_loss: 0.7624 - val_accuracy: 0.5928


In [252]:
batch_size = 32
nb_epochs = 10

hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs, validation_data=(X_test, y_test))

model.save('model1.hdf5')

Epoch 1/10
171/171 [==============================] - 12s 68ms/step - loss: 0.1848 - accuracy: 0.9649 - val_loss: 0.2196 - val_accuracy: 0.9737
Epoch 2/10
171/171 [==============================] - 12s 70ms/step - loss: 0.2002 - accuracy: 0.9527 - val_loss: 0.2088 - val_accuracy: 0.9759
Epoch 3/10
171/171 [==============================] - 12s 70ms/step - loss: 0.1967 - accuracy: 0.9642 - val_loss: 0.2169 - val_accuracy: 0.9759
Epoch 4/10
171/171 [==============================] - 12s 70ms/step - loss: 0.1780 - accuracy: 0.9691 - val_loss: 0.2266 - val_accuracy: 0.9766
Epoch 5/10
171/171 [==============================] - 12s 70ms/step - loss: 0.1679 - accuracy: 0.9759 - val_loss: 0.2259 - val_accuracy: 0.9715
Epoch 6/10
171/171 [==============================] - 12s 70ms/step - loss: 0.1730 - accuracy: 0.9711 - val_loss: 0.2180 - val_accuracy: 0.9744
Epoch 7/10
171/171 [==============================] - 12s 70ms/step - loss: 0.1726 - accuracy: 0.9697 - val_loss: 0.2064 - val_accuracy:

In [49]:
test_values = pd.read_pickle('test_data_values')
test_labels = pd.read_pickle('test_labels')

In [50]:
test_values

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-0.901586,-0.900917,-0.900318,-0.901755,-0.901106,-0.900660,-0.900371,-0.901117,-0.901063,-0.901148,...,-0.887448,-0.887832,-0.887436,-0.886768,-0.886998,-0.887540,-0.886898,-0.886533,-0.886748,-0.886713
1,-0.887583,-0.888390,-0.888298,-0.888071,-0.887494,-0.887448,-0.887832,-0.887436,-0.886768,-0.886998,...,-0.874551,-0.873345,-0.873314,-0.873625,-0.872445,-0.873875,-0.872376,-0.873233,-0.873090,-0.873190
2,-0.874648,-0.874375,-0.874732,-0.874544,-0.873840,-0.874551,-0.873345,-0.873314,-0.873625,-0.872445,...,-0.860759,-0.859499,-0.859683,-0.860375,-0.859775,-0.859690,-0.859191,-0.859544,-0.859921,-0.859771
3,-0.859384,-0.860256,-0.860844,-0.860487,-0.859506,-0.860759,-0.859499,-0.859683,-0.860375,-0.859775,...,-0.847159,-0.847298,-0.846806,-0.846990,-0.847605,-0.847051,-0.846398,-0.847267,-0.846659,-0.846171
4,-0.847825,-0.847828,-0.847632,-0.847571,-0.847232,-0.847159,-0.847298,-0.846806,-0.846990,-0.847605,...,-0.833187,-0.833398,-0.833798,-0.833571,-0.832514,-0.832529,-0.832406,-0.832191,-0.832414,-0.831760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4095,-1.013981,-0.994308,-0.993894,-0.990793,-1.019035,-0.996026,-0.983637,-0.996609,-0.996955,-1.011656,...,-0.987845,-0.995404,-1.004695,-0.993447,-1.003347,-0.999714,-1.011468,-1.001956,-1.000227,-0.991298
4096,-1.016446,-0.994438,-1.004724,-1.010113,-1.007526,-0.987845,-0.995404,-1.004695,-0.993447,-1.003347,...,-0.991836,-1.004428,-1.000935,-1.008884,-1.007629,-1.011201,-1.005180,-1.012828,-1.008138,-1.000120
4097,-0.989538,-1.015580,-1.002616,-0.996496,-1.004726,-0.991836,-1.004428,-1.000935,-1.008884,-1.007629,...,-1.020212,-1.002727,-1.014782,-1.004433,-0.989422,-1.005432,-1.015047,-0.993692,-1.003962,-0.999594
4098,-0.995656,-1.003081,-1.001588,-1.002686,-0.993313,-1.020212,-1.002727,-1.014782,-1.004433,-0.989422,...,-1.008192,-1.007724,-0.999536,-1.002337,-0.999768,-0.997948,-1.003441,-1.008146,-1.010657,-1.001110


In [51]:
test_values = test_values.values
test_values = test_values.reshape((test_values.shape[0], test_values.shape[1], 1))

In [53]:
test_values.shape

(4100, 150, 1)

In [253]:
predictions = model.predict(test_values)


129/129 [==============================] - 3s 22ms/step


In [254]:
predictions

array([[9.9985027e-01, 1.2970601e-04, 0.0000000e+00, 2.6459085e-10,
        8.0338548e-12, 1.9981904e-05],
       [9.9703169e-01, 2.9662673e-03, 0.0000000e+00, 3.0329730e-12,
        8.1376648e-11, 1.9991078e-06],
       [3.6086941e-01, 6.3913035e-01, 0.0000000e+00, 2.1593414e-13,
        4.1875867e-09, 2.2367858e-07],
       ...,
       [2.4517945e-09, 1.5257289e-05, 1.4485131e-04, 1.8355233e-05,
        1.8492981e-11, 9.9982160e-01],
       [1.2897338e-09, 2.8804047e-06, 8.1640216e-08, 8.6834768e-07,
        6.0279407e-12, 9.9999619e-01],
       [4.3627072e-08, 2.0809578e-11, 0.0000000e+00, 0.0000000e+00,
        7.1268962e-13, 1.0000000e+00]], dtype=float32)

In [255]:
prediction_list = []

threshold = 0.05

# Iterate over each row of predictions
for row in predictions:
    # Sort the probabilities in descending order and get the indices
    sorted_indices = np.argsort(row)[::-1]

    # Check if the highest prediction is above the threshold
    if row[sorted_indices[0]] >= threshold:
        prediction1 = sorted_indices[0]
    else:
        prediction1 = 3

    # Check if the second highest prediction is above the threshold
    if row[sorted_indices[1]] >= threshold:
        prediction2 = sorted_indices[1]
    else:
        prediction2 = 3

    # Add the predictions to the list
    prediction_list.append([prediction1, prediction2])

# Create the DataFrame from the list
#df = pd.DataFrame(prediction_list, columns=['prediction1', 'prediction2'])
prediction_list = np.array(prediction_list)
labels = ['Bias', 'Drift', 'Gain', 'NoFault', 'Outliers', 'Precisiondegradation']
label_encoder = LabelEncoder()
label_encoder.fit(labels)
predicted_labels_1 = label_encoder.inverse_transform(prediction_list[:,0])
predicted_labels_2 = label_encoder.inverse_transform(prediction_list[:,1])
df_predictions = pd.DataFrame({'Prediction': predicted_labels_1, 'Prediction2': predicted_labels_2})

# Display the resulting DataFrame
print(df_predictions)
df_predictions.to_excel('predictions1.xlsx')



                Prediction Prediction2
0                     Bias     NoFault
1                     Bias     NoFault
2                    Drift        Bias
3                    Drift     NoFault
4                    Drift     NoFault
...                    ...         ...
4095  Precisiondegradation     NoFault
4096  Precisiondegradation     NoFault
4097  Precisiondegradation     NoFault
4098  Precisiondegradation     NoFault
4099  Precisiondegradation     NoFault

[4100 rows x 2 columns]


In [135]:
top_two_probs = np.sort(predictions, axis=1)[:, -2:]
top_two_probs

array([[0.0171706 , 0.9817509 ],
       [0.02506384, 0.9733617 ],
       [0.03759037, 0.95918125],
       ...,
       [0.32750604, 0.45511836],
       [0.34040767, 0.47297347],
       [0.04950599, 0.942229  ]], dtype=float32)

In [245]:
top_two_indices = np.argsort(predictions, axis=1)[:, -2:]
predicted_label_indices = top_two_indices[:, ::-1]
first_predictions = predicted_label_indices[:,0]
second_predictions = predicted_label_indices[:,1]
labels = ['Bias', 'Drift', 'Gain', 'NoFault', 'Outliers', 'Precisiondegradation']
label_encoder = LabelEncoder()
label_encoder.fit(labels)
predicted_labels_1 = label_encoder.inverse_transform(first_predictions)
predicted_labels_2 = label_encoder.inverse_transform(second_predictions)
df_predictions = pd.DataFrame({'Prediction': predicted_labels_1, 'Prediction2': predicted_labels_2})

df_predictions


,Prediction,Prediction2
0,Bias,Drift
1,Bias,Drift
2,Drift,Bias
3,Drift,Bias
4,Drift,Bias
...,...,...
4095,Precisiondegradation,Gain
4096,Precisiondegradation,Gain
4097,Precisiondegradation,Gain
4098,Precisiondegradation,Gain


In [195]:
df_predictions.to_excel('predictions.xlsx')

In [180]:
labels = ['Bias', 'Drift', 'Gain', 'NoFault', 'Outliers', 'Precisiondegradation']
predicted_labels = np.argmax(predictions, axis=1)
predicted_label_indices = np.argmax(predictions, axis=1)

# top_two_indices = np.argsort(predictions, axis=1)[:, -2:]
# predicted_label_indices = top_two_indices[:, ::-1]

label_encoder = LabelEncoder()
label_encoder.fit(labels)

predicted_labels = label_encoder.inverse_transform(predicted_label_indices)

predicted_labels

array(['Bias', 'Bias', 'Bias', ..., 'Gain', 'Gain', 'NoFault'],
      dtype='<U20')

In [101]:
top_two_probs = np.sort(predictions, axis=1)[:, -2:]
threshold = 0.5  # Adjust this threshold according to your needs

predicted_labels = np.zeros((predictions.shape[0], 2), dtype=np.int32)
predicted_labels[:, 0] = predicted_label_indices

mask = top_two_probs[:, 1] > threshold
predicted_labels[mask, 1] = predicted_label_indices[mask]


In [181]:
top_two_probs = np.sort(predictions, axis=1)[:, -2:]
threshold = 0.5  # Adjust this threshold according to your needs

predicted_labels = np.zeros((predictions.shape[0], 1), dtype=np.int32)
predicted_labels[:, 0] = predicted_label_indices[:predicted_labels.shape[0]]

mask = top_two_probs[:, 1] > threshold

predicted_labels_2 = np.zeros((predicted_labels.shape[0], 1), dtype=np.int32)
predicted_labels_2[mask] = predicted_label_indices[mask].reshape(-1, 1)

predicted_labels = np.concatenate((predicted_labels, predicted_labels_2), axis=1)


In [182]:
prediction_column = label_encoder.inverse_transform(predicted_labels[:, 0])
prediction2_column = label_encoder.inverse_transform(predicted_labels[:, 1])

predictions_df = pd.DataFrame({'Prediction': prediction_column, 'Prediction2': prediction2_column})
print(predictions_df)



     Prediction Prediction2
0          Bias        Bias
1          Bias        Bias
2          Bias        Bias
3       NoFault        Bias
4       NoFault        Bias
...         ...         ...
4095       Gain        Bias
4096       Gain        Bias
4097       Gain        Bias
4098       Gain        Bias
4099    NoFault        Bias

[4100 rows x 2 columns]


In [115]:
predictions_df.to_excel('predictions.xlsx')

In [114]:
predicted_label_indices

array([0, 0, 0, ..., 3, 3, 0], dtype=int64)

In [78]:
predicted_labels_df = pd.DataFrame(predicted_labels)
predicted_labels_df.to_excel('predictions.xlsx')